In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from sklearn.metrics import roc_auc_score

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/11.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/12.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112400567,ow5aQaW,0
1,1112306051,lDLm5EG,0
2,1111609950,xkd0Ebz,0
3,1112289649,zv43m38,0
4,1112297123,1YqKoR,0
5,1112348520,Nz0K25B,0
6,1112289881,ZDrda4E,0
7,1111954700,Z93Pdb,0
8,1112423684,VNPJGEm,0
9,1112295743,eJdZz9,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16502031


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
9149524,1112316661,aEBdKE,1,FEM,29.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
4710848,1112277266,vVj0jvj,1,MASC,24.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,True,False,True,NaN
9803710,1110728205,Bm5b59Z,0,MASC,22.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,False,False,False,NaN
1778617,1112305277,qe2zMRL,1,FEM,23.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,False,NaN
11631199,1112433020,8z5d5l,1,MASC,36.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.010753
275948,1112350129,Ezea336,0,FEM,45.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,False,NaN
5170417,1112319196,jkdV6ar,0,MASC,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
7144001,1112363352,xkdJ8Ea,1,MASC,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
4161012,1112383501,VNJrjMm,1,FEM,41.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,0.022989
13450072,1111909764,ow2qGx0,0,FEM,22.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
640964,1112260321,GNZX4G8,1,FEM,19.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
7917726,1112385028,5l8LKN,0,FEM,37.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.000000
16003874,1483330,X9PXlp8,0,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
9135822,1112456018,a1k3aD,0,MASC,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
15029036,1112285533,JBeRkee,0,FEM,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
2491043,1112295409,xk9O4zb,0,FEM,25.0,-,-,-,-,Graduado,...,False,False,True,False,False,True,False,False,False,0.000000
6424463,1112345265,5mlKkXq,1,MASC,25.0,-,-,-,-,Graduado,...,False,True,True,False,False,False,False,False,False,0.010753
15540032,1112473400,mzdrRVO,0,MASC,26.0,-,-,-,-,Graduado,...,False,False,False,True,True,False,False,False,False,0.000000
14006451,1112483647,QNMbE6e,0,FEM,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
1241062,1112202162,Yj5m13w,0,MASC,28.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.000000


In [15]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  15281897 Test:  804311


In [16]:
gc.collect()

161

In [17]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=6000,max_depth=13,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.690161
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's binary_logloss: 0.68628
[3]	valid_0's binary_logloss: 0.682237
[4]	valid_0's binary_logloss: 0.679089
[5]	valid_0's binary_logloss: 0.675165
[6]	valid_0's binary_logloss: 0.671842
[7]	valid_0's binary_logloss: 0.668148
[8]	valid_0's binary_logloss: 0.664831
[9]	valid_0's binary_logloss: 0.663312
[10]	valid_0's binary_logloss: 0.660494
[11]	valid_0's binary_logloss: 0.65713
[12]	valid_0's binary_logloss: 0.653781
[13]	valid_0's binary_logloss: 0.651323
[14]	valid_0's binary_logloss: 0.649926
[15]	valid_0's binary_logloss: 0.647477
[16]	valid_0's binary_logloss: 0.644454
[17]	valid_0's binary_logloss: 0.641193
[18]	valid_0's binary_logloss: 0.638859
[19]	valid_0's binary_logloss: 0.635986
[20]	valid_0's binary_logloss: 0.632818
[21]	valid_0's binary_logloss: 0.62974
[22]	valid_0's binary_logloss: 0.627629
[23]	valid_0's binary_logloss: 0.62529
[24]	valid_0's binary_logloss: 0

In [19]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9018740287964169


In [20]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV21.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV21.pkl']

In [21]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
22329,14504,1112094756,ZDr50Nb,FEM,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
54479,42830,1112423965,jkV1Zle,FEM,30.0,-,-,-,En Curso,Graduado,...,False,False,True,False,True,True,False,False,False,0.047619
46435,67207,1112460187,ek4Elq9,MASC,51.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.103448
90158,55358,1112447611,a5E1qW,MASC,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
82143,30103,1112377235,Z0XAB1,MASC,33.0,-,-,-,-,-,...,False,False,False,True,False,False,False,True,False,0.000000
25810,86993,1112466773,owjd2Va,FEM,30.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,True,False,0.250000
86275,47464,1112431352,e8RBEB,FEM,31.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
87728,41885,1112422083,8LNpLR,FEM,33.0,-,-,-,En Curso,-,...,False,False,False,False,False,False,False,False,False,0.000000
52756,32590,1112395182,eeEXO4,MASC,34.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.000000
11432,36947,1112412660,vV93dX1,FEM,26.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.038462


In [22]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.88593574, 0.11406426],
       [0.86106207, 0.13893793],
       [0.84173042, 0.15826958],
       ...,
       [0.97403866, 0.02596134],
       [0.96794249, 0.03205751],
       [0.99467596, 0.00532404]])

In [23]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [24]:
prediccion['sepostulo']=y_final2

In [25]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.114064
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.138938
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.158270
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.029151
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.041866


In [26]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,0.114064
1,1,0.138938
2,2,0.158270
3,3,0.029151
4,4,0.041866


In [27]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion45.csv",index=False)

In [28]:
prediccion.sepostulo.mean()

0.5334644976406802